# Classify Fluorescence Images as Polarized

This notebook implements a basic CNN to classify fluorescent images of embryos as having or lacking polarized caps.

In [1]:
!pip install mxnet

In [2]:
from __future__ import print_function
import h5py
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
import utils
import os
import numpy as np

# Fixing the random seed
mx.random.seed(42)

### Define Model

In [3]:
import mxnet.ndarray as F

class Net(gluon.Block):
    def __init__(self, **kwargs):
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            # layers created in name_scope will inherit name space
            # from parent layer.
            self.conv1 = nn.Conv2D(20, kernel_size=(5,5))
            self.pool1 = nn.MaxPool2D(pool_size=(2,2), strides = (2,2))
            self.conv2 = nn.Conv2D(50, kernel_size=(5,5))
            self.pool2 = nn.MaxPool2D(pool_size=(2,2), strides = (2,2))
            self.fc1 = nn.Dense(500)
            self.fc2 = nn.Dense(2)

    def forward(self, x):
        x = self.pool1(F.tanh(self.conv1(x)))
        x = self.pool2(F.tanh(self.conv2(x)))
        # 0 means copy over size from corresponding dimension.
        # -1 means infer size from the rest of dimensions.
        x = x.reshape((0, -1))
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        return x

In [4]:
net = Net()

### Data Preprocessing

In [5]:
# preprocess the data
fluo_data_path = "../data/video_fluo_data"
embryo_idx = 46
fluo = h5py.File(os.path.join(fluo_data_path,'embryo_'+str(embryo_idx)+'.mat'))
arrays = {}
for k, v in fluo.items():
    arrays[k] = np.array(v)

fluo_video = arrays['data']
pol_state = arrays['anno']

# a bit of data processing... get middle z slice
fluo_video = np.array([utils.get_middle_z(fluo_video)])
fluo_video = np.moveaxis(fluo_video, -1, 0)

# # train-test split
# from sklearn.model_selection import train_test_split
# # split 0.2 for test set
# X_train, X_test, y_train, y_test = train_test_split(fluo_video, pol_state[0], test_size=0.2, random_state=42)
# # split 0.2 for val set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# # keep 0.6 for train set

### Data Loaders

In [6]:
# # define the batch size for data loading
# batch_size = 30

# # data loaders?
# # iterator vs dataloader: https://mxnet.apache.org/versions/1.7/api/python/docs/tutorials/packages/gluon/data/datasets.html#Appendix:-Upgrading-from-Module-DataIter-to-Gluon-DataLoader
# # dataloader: https://mxnet.apache.org/versions/1.7/api/python/docs/tutorials/packages/gluon/data/datasets.html
# # data iterator: https://mxnet.apache.org/versions/1.1.0/tutorials/basic/data.html
# train_data = mx.io.NDArrayIter(X_train, y_train, batch_size=batch_size, shuffle=True)
# test_data = mx.io.NDArrayIter(X_test, y_test, batch_size=batch_size)
# val_data = mx.io.NDArrayIter(X_val, y_val, batch_size=batch_size)
# # # Convert to dataloader:
# # data_iter_loader = DataIterLoader(data_iter)
# # for X_batch, y_batch in data_iter_loader:

In [7]:
# define the batch size for data loading
batch_size = 30

train_data = mx.io.NDArrayIter(fluo_video, pol_state[0], batch_size=batch_size, shuffle=True)

### Optimizer, Metrics, Trainer

In [8]:
# set the context on GPU is available otherwise CPU
ctx = [mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()]
net.initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

# For training
# Use Accuracy as the evaluation metric.
metric = mx.metric.Accuracy()
softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()

# For validation/test
# define an evaluation function for validation and testing
def test(net, test_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(test_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)
    return metric.get()

# learning rate
lr = 0.001
# lr_factor = 0.75 # Learning rate decay factor
# lr_steps = [10, 20, 30, np.inf] # Epochs where learning rate decays
wd = 0.0001
momentum = 0.0
optimizer = 'sgd' # 'nag': Nesterov accelerated gradient descent
optimizer_params = {'learning_rate': lr, 'wd': wd, 'momentum': momentum} # Set parameters
trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)

### Training Loop

In [9]:
epochs = 10

# lr_lst = [0.001, 0.005, 0.01, 0.05, 0.1]
# low learning rate, high momentum
lr_lst = [0.02]

train_acc_over_hyperparam_tensor = []
# val_acc_over_hyperparam_tensor = []

for lr_val in lr_lst:
    # set learning rate
    trainer.set_learning_rate(lr_val)
    
    # store results
    train_acc_lst = np.zeros(epochs)
#     val_acc_lst = np.zeros(epochs)

    for i in range(epochs):
        # Reset the train data iterator.
        train_data.reset()
        # Loop over the train data iterator.
        for batch in train_data:
            # Splits train data into multiple slices along batch_axis
            # and copy each slice into a context.
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            # Splits train labels into multiple slices along batch_axis
            # and copy each slice into a context.
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            # Inside training scope
            with ag.record():
                for x, y in zip(data, label):
                    z = net(x)
                    # Computes softmax cross entropy loss.
                    loss = softmax_cross_entropy_loss(z, y)
                    # Backpropogate the error for one iteration.
                    loss.backward()
                    outputs.append(z)

            print(label, outputs)
            # Updates internal evaluation
            metric.update(label, outputs)
            # Make one step of parameter update. Trainer needs to know the
            # batch size of data to normalize the gradient by 1/batch_size.
            trainer.step(batch.data[0].shape[0])

        # Gets the training evaluation result.
        name, train_acc = metric.get()
        train_acc_lst[i] = train_acc

    #     # Gets the validation evaluation result.
    #     name, val_acc = test(net,val_data,ctx)
    #     val_acc_lst[i] = val_acc

        # Reset evaluation result to initial state.
        metric.reset()
        print('training acc at epoch %d: %s=%f'%(i, name, train_acc))
    
    train_acc_over_hyperparam_tensor.append(train_acc_lst)
#     val_acc_over_hyperparam_tensor.append(val_acc_lst)

[
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0.]
<NDArray 30 @cpu(0)>] [
[[ 0.14373255  0.36559972]
 [-0.27026007  0.35968503]
 [ 0.02855117  0.4424781 ]
 [-0.21087226  0.267378  ]
 [ 0.12967221  0.37347943]
 [-0.21080026  0.5813768 ]
 [-0.01696655  0.36416656]
 [ 0.32669455  0.04036618]
 [-0.10103721  0.46833882]
 [ 0.17513482  0.60633945]
 [ 0.01648585  0.29663554]
 [ 0.20187     0.4214257 ]
 [-0.0731499   0.33264264]
 [ 0.32309768 -0.08252288]
 [ 0.16820839  0.18376893]
 [-0.10447828  0.25685436]
 [-0.07699339  0.49889588]
 [-0.04840959  0.52380294]
 [ 0.25001052  0.26115426]
 [ 0.05989517  0.53913635]
 [-0.07991438  0.09435862]
 [ 0.14373255  0.36559972]
 [-0.27026007  0.35968503]
 [ 0.02855117  0.4424781 ]
 [-0.21087226  0.267378  ]
 [ 0.12967221  0.37347943]
 [-0.21080026  0.5813768 ]
 [-0.01696655  0.36416656]
 [ 0.32669455  0.04036618]
 [-0.10103721  0.46833882]]
<NDArray 30x2 @cpu(0)>]
training acc at epoch 0: accuracy=0.233333
[
[

KeyboardInterrupt: 

### Plot Results

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(np.arange(0,epochs,1),train_acc_over_hyperparam_tensor[0],'r')
plt.ylabel('Training Accuracy')
plt.xlabel('Epoch')
plt.title('CNN Classify Fluo Polarization')
plt.show()